In [1]:
import requests
import pandas as pd
import sqlite3
import time as tm
from datetime import datetime, timedelta
db_path = r'D:\Rfiles\量化投资学习\database\high_freq.db'

# 库表结构和数据类型

In [2]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 遍历表名并获取每个表的列信息
for table in tables:
    table_name = table[0]
    print("Table: ", table_name)

    cursor.execute("SELECT COUNT(*) FROM {}".format(table_name))
    row_count = cursor.fetchone()[0]
    print("Row count: ", row_count)

    cursor.execute("PRAGMA table_info({})".format(table_name))
    columns = cursor.fetchall()
    for column in columns:
        column_name = column[1]
        column_type = column[2]
        print("Column: {}, Type: {}".format(column_name, column_type))

    print("\n")
conn.close()


Table:  Popular_stock
Row count:  3187225
Column: time, Type: TEXT
Column: thscode, Type: TEXT
Column: open, Type: REAL
Column: high, Type: REAL
Column: low, Type: REAL
Column: close, Type: REAL
Column: avgPrice, Type: REAL
Column: volume, Type: REAL
Column: amount, Type: REAL
Column: change, Type: REAL
Column: changeRatio, Type: REAL
Column: turnoverRatio, Type: REAL
Column: sellVolume, Type: INTEGER
Column: buyVolume, Type: INTEGER
Column: BBI, Type: REAL
Column: MA, Type: REAL
Column: MACD, Type: REAL
Column: TRIX, Type: REAL
Column: RSI, Type: REAL
Column: ARBR, Type: REAL
Column: BOLL, Type: REAL
Column: VOSC, Type: REAL
Column: ZDZB, Type: REAL


Table:  limit_up_stock
Row count:  3543905
Column: time, Type: TEXT
Column: thscode, Type: TEXT
Column: open, Type: REAL
Column: high, Type: REAL
Column: low, Type: REAL
Column: close, Type: REAL
Column: avgPrice, Type: REAL
Column: volume, Type: REAL
Column: amount, Type: REAL
Column: change, Type: REAL
Column: changeRatio, Type: REAL
C

In [2]:
conn = sqlite3.connect(db_path)

df = pd.read_sql("SELECT l.thscode, s.SECURITY_NAME, min(l.time) FROM zhangtinggu l join Security_name s on l.thscode = s.THSCODE group by 1,2",conn)
conn.close()

df

,thscode,SECURITY_NAME,min(l.time)
0,000004.SZ,国华网安,2023-08-21 09:30
1,000010.SZ,美丽生态,2023-08-21 09:30
2,000023.SZ,ST深天,2023-08-21 09:30
3,000025.SZ,特力A,2023-08-21 09:30
4,000046.SZ,*ST泛海,2023-08-21 09:30
...,...,...,...
309,688356.SH,键凯科技,2023-08-21 09:30
310,688418.SH,震有科技,2023-08-21 09:30
311,688523.SH,航天环宇,2023-08-21 09:30
312,688629.SH,华丰科技,2023-08-21 09:30


In [6]:
conn = sqlite3.connect(db_path)
df = pd.read_sql("""WITH RankedData AS (
  SELECT
    time,
    thscode,
    open,
    high,
    low,
    close,
    avgPrice,
    volume,
    amount,
    change,
    changeRatio,
    turnoverRatio,
    sellVolume,
    buyVolume,
    BBI,
    MA,
    MACD,
    TRIX,
    RSI,
    ARBR,
    BOLL,
    VOSC,
    ZDZB,
    ROW_NUMBER() OVER (PARTITION BY time, thscode) AS rn
  FROM zhangtinggu
)
SELECT
  time,
  thscode,
  open,
  high,
  low,
  close,
  avgPrice,
  volume,
  amount,
  change,
  changeRatio,
  turnoverRatio,
  sellVolume,
  buyVolume,
  BBI,
  MA,
  MACD,
  TRIX,
  RSI,
  ARBR,
  BOLL,
  VOSC,
  ZDZB
FROM RankedData
WHERE rn = 1;""",conn)

conn.close()

df

,time,thscode,open,high,low,close,avgPrice,volume,amount,change,...,buyVolume,BBI,MA,MACD,TRIX,RSI,ARBR,BOLL,VOSC,ZDZB
0,2023-08-21 09:30,000004.SZ,16.64,16.64,16.64,16.64,16.64,164200.0,2732288.00,-0.01,...,0.0,16.694063,16.650,-0.038941,-0.024234,14.903698,115.000000,16.624490,19.866491,0.876979
1,2023-08-21 09:30,000010.SZ,3.11,3.11,3.11,3.11,3.11,2649500.0,8239945.00,-0.05,...,0.0,3.152188,3.150,-0.003960,-0.005760,0.000002,-2005.000000,3.138846,44.683514,16.410714
2,2023-08-21 09:30,000023.SZ,6.77,6.77,6.77,6.77,6.77,12700.0,85979.00,-0.01,...,0.0,6.776354,6.772,-0.007313,-0.017210,40.496805,133.333333,6.760962,1.726079,2.981855
3,2023-08-21 09:30,000025.SZ,16.02,16.02,16.02,16.02,16.02,2900.0,46458.00,0.01,...,1900.0,16.028125,16.024,-0.011915,-0.008182,37.696108,112.500000,16.016340,15.876182,1.432127
4,2023-08-21 09:30,000046.SZ,1.75,1.75,1.75,1.75,1.75,18173423.0,31803490.25,-0.09,...,0.0,1.825938,1.822,-0.015414,-0.060576,0.139024,-2005.000000,1.801923,43.282051,14.277778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534260,2023-11-01 15:00,688356.SH,126.07,126.07,126.07,126.07,126.07,21684.0,2733701.88,0.15,...,0.0,125.857187,125.922,0.051080,0.004481,67.978475,174.647887,125.208229,17.201635,1.777778
3534261,2023-11-01 15:00,688418.SH,22.28,22.28,22.28,22.28,22.28,213291.0,4752120.00,0.00,...,121755.0,22.236146,22.266,0.015728,0.011426,71.085213,113.793103,22.066004,10.045860,1.009938
3534262,2023-11-01 15:00,688523.SH,31.15,31.15,31.15,31.15,31.15,202596.0,6310865.40,0.15,...,0.0,30.730104,31.004,0.366197,0.177913,82.047967,188.414634,29.045335,36.819059,1.063062
3534263,2023-11-01 15:00,688629.SH,24.02,24.02,24.02,24.02,24.02,49934.0,1199414.68,0.02,...,0.0,24.011146,24.010,-0.013364,-0.011226,56.291285,121.212121,23.940373,18.691865,1.192982
